## Importe de bibliotecas

In [ ]:
import multiprocessing

from gensim.models import Word2Vec

In [ ]:
import pandas as pd

## Carga de datos

In [ ]:
resumenes_docentes = pd.read_csv("../ResumenesDocentes.csv")
resumenes_estudiantes = pd.read_csv("../ResumenesEstudiantes.csv")

## Verificacion datos

In [ ]:
resumenes_docentes.shape

In [ ]:
resumenes_docentes.head(3)

In [ ]:
resumenes_estudiantes.shape

In [ ]:
resumenes_estudiantes.head(3)

## Bigramas

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
phrases = Phrases(lemas_fila1_spliteado, min_count=30, progress_per=10000)

In [ ]:
bigram = Phraser(phrases)

In [ ]:
sentences = bigram[lemas_fila1_spliteado]

## Word2vec

In [ ]:
lemas_corpus=resumenes_docentes['lemas_corpus']

In [ ]:
lemas_fila1=lemas_corpus[0]

In [ ]:
lemas_fila1_spliteado=lemas_fila1.split()

In [ ]:
lemas_fila1_spliteado

In [ ]:
modelo = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=7)

In [ ]:
modelo

In [ ]:
# t = time()

modelo.build_vocab([lemas_fila1], progress_per=10000)

# print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
# t = time()

modelo.train([lemas_fila1], total_examples=modelo.corpus_count, epochs=30, report_delay=1)

# print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
# w2v_model.init_sims(replace=True)

In [ ]:
# modelo_visualizar = pd.DataFrame(modelo.toarray(), columns=modelo.get_feature_names())

# display(modelo_visualizar)
# display(df_estructurado_tf_idf.iloc[: ,1000:1500])
# display(df_complete['des'].iloc[3])

## Exploración de modelo

In [ ]:
modelo.wv.most_similar(positive=["d"])

In [ ]:
modelo.most_similar(positive=["i"])